In [36]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
fn_out = '/Volumes/Extreme SSD/energy_communities/clean_input/geography/edd_to_census_tract.parquet'
target_epsg = 3857

In [7]:
EddRaw = gpd.read_file('/Volumes/Extreme SSD/energy_communities/raw_input/EDD Layer - October 2024', encoding='utf-8')
EddRaw.set_geometry('geometry', inplace=True)
EddRaw.to_crs(epsg=target_epsg, inplace=True)
# Edd = EddRaw.to_crs(epsg=4269, inplace=False)
Edd = EddRaw[['FullName', 'geometry']]
Edd = Edd.rename(columns={'FullName':'edd_full_name', 'geometry':'edd_geometry'})
Edd = Edd.set_geometry('edd_geometry')
Edd.head()

,edd_full_name,edd_geometry
0,3Core Economic Development Corporation (CA)/Tr...,"POLYGON ((-13539095.212 4764106.072, -13539103..."
1,Acadiana Planning Commission,"POLYGON ((-10225791.130 3421410.172, -10226779..."
2,Accomack-Northampton Planning District Commission,"POLYGON ((-8475766.200 4504656.331, -8475838.5..."
3,Alabama-Tombigbee Regional Commission,"POLYGON ((-9846562.853 3698558.404, -9846576.2..."
4,Alamo Area Council of Governments,"POLYGON ((-10998736.257 3256240.291, -10998735..."


In [9]:
# Make sure there is one row for each EDD
Edd[['edd_full_name']].duplicated().sum() == 0

True

In [10]:
census_dirs = '/Volumes/Extreme SSD/energy_communities/raw_input/census_tract/2023'

In [16]:
list_of_census_files = [ f.path for f in os.scandir(census_dirs) if f.is_dir() ]
print(list_of_census_files[:3])

['/Volumes/Extreme SSD/energy_communities/raw_input/census_tract/2023/tl_2023_04_tract', '/Volumes/Extreme SSD/energy_communities/raw_input/census_tract/2023/tl_2023_02_tract', '/Volumes/Extreme SSD/energy_communities/raw_input/census_tract/2023/tl_2023_01_tract']


In [35]:
results_dict = {}
for e in Edd.index:
    results_dict[e] = list()

In [38]:
results_dict = {}
for e in Edd.index:
    results_dict[e] = list()
    
for census_subfolder in tqdm(list_of_census_files):
    CensusObservation = gpd.read_file(census_subfolder)
    CensusObservation = CensusObservation.rename(columns={'GEOID':'census_geoid', 'GEOIDFQ':'census_geoidfq', 'geometry':'census_geometry'})
    CensusObservation = CensusObservation[['census_geoid', 'census_geoidfq', 'census_geometry']]
    CensusObservation = CensusObservation.set_geometry('census_geometry')
    CensusObservation = CensusObservation.to_crs(epsg=target_epsg)

    for e in Edd.index:
        EddDatum = Edd.loc[[e]]
        Intersection = gpd.sjoin(EddDatum, CensusObservation, how='inner', predicate='intersects')
        if Intersection.shape[0] == 0:
            pass
        else:
            results_dict[e].append(Intersection)

  0%|          | 0/56 [00:00<?, ?it/s]

In [61]:
Results = pd.DataFrame()
for k in results_dict.keys():
    Concat = pd.concat(results_dict[k], axis=0, ignore_index=False)
    Concat = Concat[['edd_full_name', 'census_geoid', 'census_geoidfq']]
    Concat.drop_duplicates(inplace=True)
    Concat['edd_index'] = k
    Results = pd.concat([Results, Concat], axis=0, ignore_index=True)
Results = Results[['edd_index', 'edd_full_name', 'census_geoid', 'census_geoidfq']]

In [62]:
Results.to_parquet(fn_out)